In [ ]:
import csv
import logging
from db import DB

# Initialize your database connection (assuming DB is your ORM class)
db = DB()

csv_file = 'data/us_gaap_2025_description_variations.csv'

def load_csv(file_path):
    """
    Loads the data from a CSV file.

    Parameters:
    - file_path (str): Path to the CSV file.

    Returns:
    - list: Parsed data from the CSV file as a list of dictionaries.
    """
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            data.append(row)
    return data

def upsert_description_variations(db, csv_data):
    """
    Inserts GAAP concept description variations into the database.

    Parameters:
    - db: The DB instance from your ORM.
    - csv_data (list): The parsed CSV data.
    """
    try:
        for row in csv_data:
            concept_name = row['tag']
            variation_text = row['description_variation'].lower()

            if not concept_name or not variation_text:
                continue

            # Look up the concept ID
            concept_row = db.get(
                "SELECT id FROM us_gaap_concept WHERE name = %s",
                ["id"],
                params=(concept_name,)
            )

            if concept_row.empty:
                logging.warning("Skipping unknown concept: %s", concept_name)
                continue

            concept_id = concept_row.iloc[0]['id']

            # Insert variation (no deduplication enforced by schema)
            db.upsert_entity(
                table_name="us_gaap_concept_description_variation",
                field_dict={
                    'us_gaap_concept_id': concept_id,
                    'text': variation_text
                },
                unique_fields=["us_gaap_concept_id", "text"]
            )

            logging.debug("Inserted variation for concept: %s", concept_name)

        logging.info("All description variations imported successfully.")

    except Exception as e:
        logging.error(f"Error importing description variations: {e}")
        raise

if __name__ == "__main__":
    csv_data = load_csv(csv_file)
    upsert_description_variations(db, csv_data)
